In [1]:
!pip install streamlit
!pip install pyngrok
from pyngrok import ngrok

     |████████████████████████████████| 8.2 MB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 4.2 MB 54.2 MB/s eta 0:00:01
     |████████████████████████████████| 74 kB 3.5 MB/s  eta 0:00:01
     |████████████████████████████████| 745 kB 4.4 MB/s eta 0:00:01
  Created wheel for pyngrok: filename=pyngrok-5.0.5-py3-none-any.whl size=19246 sha256=6f6b2d9b9f45e4419c302d7b348259664e540a646784af8937368b0424811e1f
  Stored in directory: /root/.cache/pip/wheels/1b/f7/72/35c95a53d15b91dd00df6cf1304d49a31ec5ed6f954c2d4e32
Successfully built pyngrok


In [2]:
!ngrok authtoken 1sr9w3cbyDhpCts7ZQSd9LN2n0Z_2QQRHmgb4dKgcd5LwwHrd

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml                                      


In [3]:
%%writefile SessionState.py
try:
    import streamlit.ReportThread as ReportThread
    from streamlit.server.Server import Server
except Exception:
    # Streamlit >= 0.65.0
    import streamlit.report_thread as ReportThread
    from streamlit.server.server import Server


class SessionState(object):
    def __init__(self, **kwargs):
        """A new SessionState object.

        Parameters
        ----------
        **kwargs : any
            Default values for the session state.

        Example
        -------
        >>> session_state = SessionState(user_name='', favorite_color='black')
        >>> session_state.user_name = 'Mary'
        ''
        >>> session_state.favorite_color
        'black'

        """
        for key, val in kwargs.items():
            setattr(self, key, val)


def get(**kwargs):
    """Gets a SessionState object for the current session.

    Creates a new object if necessary.

    Parameters
    ----------
    **kwargs : any
        Default values you want to add to the session state, if we're creating a
        new one.

    Example
    -------
    >>> session_state = get(user_name='', favorite_color='black')
    >>> session_state.user_name
    ''
    >>> session_state.user_name = 'Mary'
    >>> session_state.favorite_color
    'black'

    Since you set user_name above, next time your script runs this will be the
    result:
    >>> session_state = get(user_name='', favorite_color='black')
    >>> session_state.user_name
    'Mary'

    """
    # Hack to get the session object from Streamlit.

    ctx = ReportThread.get_report_ctx()

    this_session = None

    current_server = Server.get_current()
    if hasattr(current_server, '_session_infos'):
        # Streamlit < 0.56
        session_infos = Server.get_current()._session_infos.values()
    else:
        session_infos = Server.get_current()._session_info_by_id.values()

    for session_info in session_infos:
        s = session_info.session
        if (
            # Streamlit < 0.54.0
            (hasattr(s, '_main_dg') and s._main_dg == ctx.main_dg)
            or
            # Streamlit >= 0.54.0
            (not hasattr(s, '_main_dg') and s.enqueue == ctx.enqueue)
            or
            # Streamlit >= 0.65.2
            (not hasattr(s, '_main_dg') and s._uploaded_file_mgr == ctx.uploaded_file_mgr)
        ):
            this_session = s

    if this_session is None:
        raise RuntimeError(
            "Oh noes. Couldn't get your Streamlit Session object. "
            'Are you doing something fancy with threads?')

    # Got the session object! Now let's attach some state into it.

    if not hasattr(this_session, '_custom_session_state'):
        this_session._custom_session_state = SessionState(**kwargs)

    return this_session._custom_session_state

Writing SessionState.py


In [63]:
%%writefile main.py
import streamlit as st
import pytesseract
from PIL import Image
from transformers import AutoModelWithLMHead, AutoTokenizer , AutoModelForQuestionAnswering ,AutoModelForSeq2SeqLM
import torch
import tensorflow
from transformers import pipeline
import SessionState
from tqdm import tqdm as tq
import cv2
import numpy as np
device=-1
state=0

st.title("Image To Text Scanner")

# Image

# Make sidebar

# MAking a function to get the text from the image
@st.cache
def image_to_text(img):
    text=np.array(img)
    newdata=pytesseract.image_to_osd(img,output_type=pytesseract.Output.DICT)
    if newdata['rotate']==90 :
        text = cv2.rotate(text, cv2.cv2.ROTATE_90_CLOCKWISE)
    elif newdata['rotate']==180:
        text=cv2.rotate(text,cv2.cv2.ROTATE_180)
    elif newdata['rotate']==270:
        text = cv2.rotate(text, cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
    return pytesseract.image_to_string(text)

def get_question(text,question_model) :
    input_question=st.text_input("Enter the question....")
    answer=question_model(context=tex,question=questions,max_ans_length=12)
    return answer['answer']

@st.cache(allow_output_mutation=True)
def call_them_once():
    # Calling all the models
    model = AutoModelForQuestionAnswering.from_pretrained("../input/question-answering-model/Question_Answering_Model")
    tokenizer = AutoTokenizer.from_pretrained("../input/question-answering-model/Question_Answering_Model")
    question_model = pipeline("question-answering",model=model,tokenizer=tokenizer,device=device)
    
    # Text summarizar model and tokenizer
    model=AutoModelForSeq2SeqLM.from_pretrained('../input/summarizer-model-base-800mb/Text Summarizer')
    tokenizer=AutoTokenizer.from_pretrained('../input/summarizer-model-base-800mb/Text Summarizer')
    summarizer_model=pipeline('summarization',model=model,tokenizer=tokenizer,device=device)
    
    # Getting the language models
    
    return question_model,summarizer_model

# Making the function to translate
# Function to translate 
@st.cache()
def translate(tex,lang1="English",lang2="Hindi"):
    path='../input/english-to-hindi-model/'+lang1+'_to_'+lang2
    model = AutoModelWithLMHead.from_pretrained(path)
    tokenizer = AutoTokenizer.from_pretrained(path)
    l1=lang1.lower()[:2]
    l2=lang2.lower()[:2]
    translation_path="translation_"+l1+"_to_"+l2
    translate=pipeline(translation_path,model=model,tokenizer=tokenizer,device=device)
    texing=tex.split('.')
    tex_total=[]
    for i in tq(texing) :
        tex_total.append(translate(i, max_length=80)[0]['translation_text'])
    translated_text='\n \n'.join(tex_total)
    return translated_text
def app(session_state):
    # Code to upload an image
    image_file=st.file_uploader("Upload Image")

    # Making the sidebar for the translation

    lang1=st.sidebar.selectbox("Origin Language ",("English","Hindi"))
    lang2=st.sidebar.selectbox("Translation Language",("Hindi","French","German"))


    # Code snippet to initiate the text
    tex=session_state.text
    input_question=None

    # Adding the session state for the question button which will be false at first
#     session_state = SessionState.get(question=False,summary=False,summary_text=None,translator_button=False,Translation=None)

    # Checking the image that was uploaded
    if image_file is not None:
        # Checking the type of the image that was uploaded
        file_details = {"Filename":image_file.name,"FileType":image_file.type,"FileSize":image_file.size}

        # Opening the image
        img = Image.open(image_file)
        im2 = np.array(img)
        session_state.image=im2
        # Converting the image to text
        tex=image_to_text(img)

#         # Showing the text on the image
#         st.markdown("The text is :"+tex)

        # Making the summary null
        session_state.summary=False

        # Showing the balloons on success
        st.balloons()

        # Showing the succes message if the text was successfully extracted
        st.success('We got the text!!')


    question_model,summarizer_model=call_them_once()
    
    def scan_image(img):
        text=np.array(img)
        newdata=pytesseract.image_to_osd(img,output_type=pytesseract.Output.DICT)
        if newdata['rotate']==90 :
            text = cv2.rotate(text, cv2.cv2.ROTATE_90_CLOCKWISE)
        elif newdata['rotate']==180:
            text=cv2.rotate(text,cv2.cv2.ROTATE_180)
        elif newdata['rotate']==270:
            text = cv2.rotate(text, cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
        img=text
        widthImg=900
        heightImg=900
        img = cv2.resize(img, (widthImg, heightImg)) # RESIZE IMAGE
        imgBlank = np.zeros((heightImg,widthImg, 3), np.uint8) # CREATE A BLANK IMAGE FOR TESTING DEBUGING IF REQUIRED
        imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # CONVERT IMAGE TO GRAY SCALE
        imgBlur = cv2.GaussianBlur(imgGray, (5, 5), 1) # ADD GAUSSIAN BLUR
#         sigma=0.33
        sigma=0.8
        v=np.median(imgBlur)
        lower = int(max(0, (1.0 - sigma) * v))
        upper = int(min(255, (1.0 + sigma) * v))
        imgThreshold = cv2.Canny(imgBlur,-1,-1) # APPLY CANNY BLUR
        kernel = np.ones((5, 5))
        imgDial = cv2.dilate(imgThreshold, kernel, iterations=2) # APPLY DILATION
        imgThreshold = cv2.erode(imgDial, kernel, iterations=1)  # APPLY EROSION

        ## FIND ALL COUNTOURS
        imgContours = img.copy() # COPY IMAGE FOR DISPLAY PURPOSES
        imgBigContour = img.copy() # COPY IMAGE FOR DISPLAY PURPOSES
        contours, hierarchy = cv2.findContours(imgThreshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # FIND ALL CONTOURS
        cv2.drawContours(imgContours, contours, -1, (0, 255, 0), 10) # DRAW ALL DETECTED CONTOURS
        def biggestContour(contours):
            biggest = np.array([])
            max_area = 0
            for i in contours:
                area = cv2.contourArea(i)
                if area > 5000:
                    peri = cv2.arcLength(i, True)
                    approx = cv2.approxPolyDP(i, 0.02 * peri, True)
                    if area > max_area and len(approx) == 4:
                        biggest = approx
                        max_area = area
            return biggest,max_area
        biggest, maxArea = biggestContour(contours)
        def reorder(myPoints):

            myPoints = myPoints.reshape((4, 2))
            myPointsNew = np.zeros((4, 1, 2), dtype=np.int32)
            add = myPoints.sum(1)

            myPointsNew[0] = myPoints[np.argmin(add)]
            myPointsNew[3] =myPoints[np.argmax(add)]
            diff = np.diff(myPoints, axis=1)
            myPointsNew[1] =myPoints[np.argmin(diff)]
            myPointsNew[2] = myPoints[np.argmax(diff)]

            return myPointsNew
        def drawRectangle(img,biggest,thickness):
            cv2.line(img, (biggest[0][0][0], biggest[0][0][1]), (biggest[1][0][0], biggest[1][0][1]), (0, 255, 0), thickness)
            cv2.line(img, (biggest[0][0][0], biggest[0][0][1]), (biggest[2][0][0], biggest[2][0][1]), (0, 255, 0), thickness)
            cv2.line(img, (biggest[3][0][0], biggest[3][0][1]), (biggest[2][0][0], biggest[2][0][1]), (0, 255, 0), thickness)
            cv2.line(img, (biggest[3][0][0], biggest[3][0][1]), (biggest[1][0][0], biggest[1][0][1]), (0, 255, 0), thickness)

            return img
        if biggest.size != 0:
            biggest=reorder(biggest)
            cv2.drawContours(imgBigContour, biggest, -1, (0, 255, 0), 20) # DRAW THE BIGGEST CONTOUR
            imgBigContour = drawRectangle(imgBigContour,biggest,2)
            pts1 = np.float32(biggest) # PREPARE POINTS FOR WARP
            pts2 = np.float32([[0, 0],[widthImg, 0], [0, heightImg],[widthImg, heightImg]]) # PREPARE POINTS FOR WARP
            matrix = cv2.getPerspectiveTransform(pts1, pts2)
            imgWarpColored = cv2.warpPerspective(img, matrix, (widthImg, heightImg))

            #REMOVE 20 PIXELS FORM EACH SIDE
            imgWarpColored=imgWarpColored[20:imgWarpColored.shape[0] - 20, 20:imgWarpColored.shape[1] - 20]
            imgWarpColored = cv2.resize(imgWarpColored,(widthImg,heightImg))

            # APPLY ADAPTIVE THRESHOLD
            imgWarpGray = cv2.cvtColor(imgWarpColored,cv2.COLOR_BGR2GRAY)
            imgAdaptiveThre= cv2.adaptiveThreshold(imgWarpGray, 255, 1, 1, 7, 2)
            imgAdaptiveThre = cv2.bitwise_not(imgAdaptiveThre)
            imgAdaptiveThre=cv2.medianBlur(imgAdaptiveThre,3)

            send_me=Image.fromarray(imgAdaptiveThre)

            return send_me

        else:
            imgGray = cv2.medianBlur(imgGray,5)

            send_me=Image.fromarray(imgGray)

            return send_me

    # Run the following code if there is text in the image
    if tex  :
        col1, col2,col3 = st.beta_columns(3)
        selle=col1.checkbox("Show text")
        selle2=col3.checkbox("Show Scanned text")

        if selle :
            # Showing the text on the image
            st.markdown("The text is :"+tex)
        if selle2 :
            st.image(scan_image(session_state.image),caption="Sea")
            
        # Make the sidebar button now for the translation since we got the text
        translater=st.sidebar.button("Translate")

        # Creating the button for the Question
        button_question = st.button("Question")

        # If the button is pressed
        if button_question or session_state.question  :
            # Changing the session state to true now
            session_state.question=True

            # Adding the text input box now
            input_question = st.text_input("Enter the question : ")

        if input_question :
            answer=question_model(context=tex,question=input_question,max_ans_length=12)
            st.markdown("The answer is : "+ answer["answer"])

        # Creating the summarizer_button 
        button_summarizer = st.button("Summarizer")

        # If the button is pressed 
        if button_summarizer or session_state.summary :
            # Changing the session state to true now
            session_state.summary=True

            if session_state.summary_text :
                pass 
            else :
                session_state.summary_text=summarizer_model(tex, min_length=12, max_length=100)[0]['summary_text']

            # Displaying the summary 
            st.markdown("Summary is : "+session_state.summary_text)
        if translater or session_state.translator_button :

            session_state.translator_button=True

            if session_state.Translation :
                pass
            else:
                with st.spinner("Wait for it...."):
                    session_state.Translation=translate(tex,lang1,lang2)

            st.markdown("The translated text is :")
            st.markdown(session_state.Translation)
    return tex

Overwriting main.py


In [64]:
%%writefile viz.py
import streamlit as st
from streamlit.proto.Selectbox_pb2 import Selectbox
import spacy
import re
import warnings
import streamlit.components.v1 as components
warnings.filterwarnings("ignore")

def app(text=None):
    
    if text :
        nlp = spacy.load("en_core_web_sm")

        # file = open('textfile.txt','r') 

        # text=file.readlines()
        # text=' '.join(text)
        text=str(text)
        main_text=text

        # Filtering the text using re
        text=re.sub(r'[^a-zA-Z ]+', '', text)

        doc=nlp(text)

        # Creating a dictionary
        dic={}

        for token in doc :
            if token.pos_ in dic :
                dic[token.pos_].append(token)
            else:
                dic[token.pos_]=[token]

        showable_content=["NOUN","VERB","ADP","ADJ","PROPN","PRON"]

        available_content=[]

        for i in showable_content :
            if i in dic :
                available_content.append(i)
            else :
                continue

        # Now we have got the available content we just need to make the side bar which will take in the content and then we can just mark them

        # These are the colors for the marking and which will show different stuff
        colors=["#e93b81","#f9b208","#afb9c8","#ededd0","#c0fefc","#f3bda1"]

        set_available_content=available_content

        add_selectbox = st.sidebar.selectbox(
            "What Grammer you wanna know from this text?",
            set_available_content
        )

        # Now we get the select box content now we can mark the values
        main_text=main_text.replace('\n',' ')

        main_text=main_text.replace('.','<br>')

        html_code="""
        <head>
        <style type="text/css">
                * {
          box-sizing: border-box;
          margin: 0;
          padding: 0;
        }

        body {
          width: 100vw;
          height: 200vh;
          padding: 1rem;
          font-family: Avenir, sans-serif;
          font-size: 112.5%;
          color: #124;
          background-image: radial-gradient(rgba(255, 0, 0, 0.3), rgba(255, 0, 0, 0) 40vw), radial-gradient(rgba(0, 128, 0, 0.3), rgba(0, 128, 0, 0) 40vw), radial-gradient(rgba(0, 0, 255, 0.3), rgba(0, 0, 255, 0) 40vw), radial-gradient(rgba(255, 255, 0, 0.3), rgba(255, 255, 0, 0) 40vw), radial-gradient(rgba(255, 0, 0, 0.3), rgba(255, 0, 0, 0) 40vw);
          background-position: -40vw 14rem, 50% 10rem, 60vw 14rem, -10vw calc(14rem + 20vw), 30vw calc(14rem + 20vw);
          background-size: 80vw 80vw;
          background-repeat: no-repeat;
        }

        nav {
          position: fixed;
          top: 0;
          right: 0;
          left: 0;
          padding: 1.5rem 1rem;
          background-color: white;
          border-top: 4px solid #c080c0;
        }

        header {
          max-width: 600px;
          margin: 24vh auto;
        }

        h1 {
          font-size: 3.2rem;
          font-weight: 900;
          line-height: 1.1;
        }
        h1 + p {
          margin-top: 1rem;
        }

        main {
          max-width: 800px;
          margin-right: auto;
          margin-left: auto;
        }

        .cards {
          display: flex;
          flex-direction: column;
          margin: -1rem;
          list-style: none;
        }
        @media (min-width: 600px) {
          .cards {
            flex-direction: row;
          }
        }

        .card {
          flex: 1;
          margin: 1rem;
          padding: 1rem;
          background-color: white;
          border-radius: 0.5rem;
          box-shadow: 0 0 6rem rgba(0, 0, 0, 0.1);
        }
        .card * {
          margin-bottom: 1rem;
        }

        .card--image {
          font-size: 6rem;
          line-height: 1;
        }
        .card.-red .card--image {
          color: #ff8080;
        }
        .card.-green .card--image {
          color: #80c080;
        }
        .card.-blue .card--image {
          color: #8080ff;
        }
            </style>
        </head>
        <body>
        <nav>Image To Text Analysis</nav>
        <header>
          <h1>Check For Syntatic Categories In Your Text</h1>
          <p>The next few lines will be the analysis</p>
        </header>
        <main>

        applexyz

        </main>
        </body>
        """
        words_to_replace=dic[add_selectbox]

        for i in words_to_replace :
            main_text=main_text.replace(str(i),"<mark>"+str(i)+"</mark>")

        html_code=html_code.replace("applexyz",main_text)

        components.html(html_code,width=1000,height=1000,scrolling=True)
    
    else :
        
        st.warning("There is no text added please go back to the image uploader and try to add some text first !!")


Overwriting viz.py


In [65]:
%%writefile word.py
import matplotlib.pyplot as plt
import wordcloud
import numpy as np
import pandas as pd
import streamlit as st
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
import nltk
from nltk.tokenize import word_tokenize as tokenizer
from nltk.probability import FreqDist
import plotly.express as px
from nltk.corpus import stopwords
stopwords=stopwords.words("english")
import re
import plotly.express as px

def plot_cloud(wordcloud):
    plt.figure(figsize=(18, 9))
    plt.imshow(wordcloud) # Display image
    plt.axis("off");  # No axis details

def app(text=None):
    if text :
        st.set_option('deprecation.showPyplotGlobalUse', False)

        col1, col2 = st.beta_columns(2)


        # Generate wordcloud
        wordcloud = WordCloud(width = 2000, height = 2000, random_state=1, background_color='black', colormap='Pastel1', collocations=False, stopwords = STOPWORDS).generate(text)
        plot_cloud(wordcloud)
        col1.pyplot()

        mask = np.array(Image.open('../input/masks/loc.png'))
        # Generate wordcloud
        wordcloud = WordCloud(width = 1000, height = 800, random_state=1, background_color='black', colormap='rainbow', collocations=False, stopwords = STOPWORDS, mask=mask).generate(text)
        plot_cloud(wordcloud)
        col2.pyplot()

        tex = re.sub(r'[^\w\s]','',text)
        text_tokens=tokenizer(tex)
        clean_tex= [w for w in text_tokens
        if w.lower() not in stopwords]

        fdist = FreqDist(clean_tex)
        common_5 = fdist.most_common(5)

        df = pd.DataFrame(common_5)
        df.columns =['Common Words', 'Count']

        fig = px.bar(df, x="Common Words", y="Count", title="5 Most Common Words", template = 'plotly_dark')
        fig.show()
        st.plotly_chart(fig)

    else :
        st.warning("There is no text added please go back to the image uploader and try to add some text first !!")


Overwriting word.py


In [66]:
%%writefile templates.py
import streamlit as st
import cv2
from PIL import Image
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import numpy as np

# Function for template 2
def function_for_old_page(tex):
    tex_max=tex.split('\n')
    lines=len(tex_max)
    if lines%5==0 :
        pass
    else:
        lines=round(lines/10)*10
    font_size=int(1+(110-lines)/5)
    # Making the word dictionary 
    word_shape={}
    cc=0
    for i in range(24,0,-1):
        word_shape[i]=cc
        cc+=5
    new_text=[]
    for i in tex_max :
        x=i
        if len(i)>word_shape[font_size] :
            while len(x)>word_shape[font_size] :
                new_text.append(x[:word_shape[font_size]])
                x=x[word_shape[font_size]:]
            new_text.append(x)
        else:
            new_text.append(i)
    lines=len(new_text)
    if lines%5==0 :
        pass
    else:
        lines=round(lines/10)*10
    font_size=int(1+(110-lines)/5)
    back=cv2.imread("../input/old-paper/old-paper-blank-page-492408.jpg")
    back=cv2.cvtColor(back,cv2.COLOR_BGR2RGB)
    img = Image.fromarray(back)
    draw = ImageDraw.Draw(img)
    font = ImageFont.truetype('../input/font-2/NexaRustSans-Black.otf', font_size)
    draw.text((50, 100), '\n'.join(new_text), (0, 0, 0), font=font , spacing=1)
    return img

def function_for_template(tex) :
    tex_frame=tex.split('\n')
    lines=len(tex_frame)
    if lines%5==0 :
        pass
    else:
        lines=round(lines/10)*10
    font_size=int(1+(90-lines)/5)
    # Making the word dictionary 
    word_shape={}
    cc=66
    for i in range(14,0,-1):
        word_shape[i]=cc
        cc+=6
    new_text=[]
    for i in tex_frame :
        x=i
        if len(i)>word_shape[font_size] :
            while len(x)>word_shape[font_size] :
                print(len(x))
                new_text.append(x[:word_shape[font_size]])
                x=x[word_shape[font_size]:]
            new_text.append(x)
        else:
            new_text.append(i)
    lines=len(new_text)
    if lines%5==0 :
        pass
    else:
        lines=round(lines/10)*10
    font_size=int(1+(90-lines)/5)
    # print(tex)
    back=cv2.imread("../input/background-image-20/blank-pages.jpg.webp")
    back=cv2.cvtColor(back,cv2.COLOR_BGR2RGB)
    img = Image.fromarray(back)
    draw = ImageDraw.Draw(img)
    font = ImageFont.truetype('../input/font-2/NexaRustSans-Black.otf', font_size)
    draw.text((124, 24), '\n'.join(new_text), (0, 0, 0), font=font , spacing=1)
    img.save("output_image.jpg")
    imagii=np.array(img)
    img=img.resize((900,900))
    return img


def app(text) :
    
    theme=st.sidebar.selectbox("Select the theme :",("Old Page","Blank Page"))
    
    if theme=="Old Page" :
        
        st.image(function_for_old_page(text),caption="Old Page Template")
        
    else :
        
        st.image(function_for_template(text),caption="Blank Page Template")


Overwriting templates.py


In [67]:
%%writefile app.py
import main
import viz
import word
import templates
import streamlit as st
import SessionState
PAGES = {
    "Main": main,
    "Visualization pt1": viz ,
    "Word" : word ,
    "Template" : templates
}
session_state = SessionState.get(question=False,summary=False,summary_text=None,
                                 translator_button=False,Translation=None,text=None,image=None)

st.sidebar.title('Navigation')
selection = st.sidebar.radio("Go to", list(PAGES.keys()))
page = PAGES[selection]
if selection in ["Visualization pt1" , "Word" , "Template"]:
    page.app(session_state.text)
elif selection=="Main" :
    session_state.text=page.app(session_state)
else :
    page.app()

Overwriting app.py


In [68]:
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(port='8501')
public_url

2021-05-22 07:48:47.458 INFO    pyngrok.ngrok: Opening tunnel named: http-80-9331fe5e-d34a-4f90-838f-57c67c66f7c9
2021-05-22 07:48:47.480 INFO    pyngrok.process.ngrok: t=2021-05-22T07:48:47+0000 lvl=info msg="no configuration paths supplied"
2021-05-22 07:48:47.483 INFO    pyngrok.process.ngrok: t=2021-05-22T07:48:47+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
2021-05-22 07:48:47.485 INFO    pyngrok.process.ngrok: t=2021-05-22T07:48:47+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
2021-05-22 07:48:47.486 INFO    pyngrok.process.ngrok: t=2021-05-22T07:48:47+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040
2021-05-22 07:48:47.613 INFO    pyngrok.process.ngrok: t=2021-05-22T07:48:47+0000 lvl=info msg="tunnel session started" obj=tunnels.session
2021-05-22 07:48:47.614 INFO    pyngrok.process.ngrok: t=2021-05-22T07:48:47+0000 lvl=info msg="client session established" obj=csess id=73f0892ef39a
2

<NgrokTunnel: "http://4986644e64b3.ngrok.io" -> "http://localhost:80">

2021-05-22 07:48:47.708 INFO    pyngrok.process.ngrok: t=2021-05-22T07:48:47+0000 lvl=info msg=end pg="/api/tunnels/http-80-9331fe5e-d34a-4f90-838f-57c67c66f7c9 (http)" id=e87f0f7ed37dcb38 status=200 dur=132.109µs


In [ ]:
!streamlit run --server.port 80 app.py >/dev/null

2021-05-22 07:48:49.358 An update to the [server] config option section was detected. To have these changes be reflected, please restart streamlit.
2021-05-22 07:48:50.308948: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


2021-05-22 07:48:53.148 INFO    pyngrok.process.ngrok: t=2021-05-22T07:48:53+0000 lvl=info msg="join connections" obj=join id=b32cbe0d5cc3 l=127.0.0.1:80 r=[2409:4050:e14:4594:7111:4dd6:3fe6:cb44]:59605
2021-05-22 07:48:53.524 INFO    pyngrok.process.ngrok: t=2021-05-22T07:48:53+0000 lvl=info msg="join connections" obj=join id=5ebaeaec5174 l=127.0.0.1:80 r=[2409:4050:e14:4594:7111:4dd6:3fe6:cb44]:59608
2021-05-22 07:48:53.537 INFO    pyngrok.process.ngrok: t=2021-05-22T07:48:53+0000 lvl=info msg="join connections" obj=join id=3537e0263a9e l=127.0.0.1:80 r=[2409:4050:e14:4594:7111:4dd6:3fe6:cb44]:59605
2021-05-22 07:48:54.948 INFO    pyngrok.process.ngrok: t=2021-05-22T07:48:54+0000 lvl=info msg="join connections" obj=join id=6c02943dc1cb l=127.0.0.1:80 r=49.44.81.38:62030
2021-05-22 07:48:56.310 INFO    pyngrok.process.ngrok: t=2021-05-22T07:48:56+0000 lvl=info msg="join connections" obj=join id=37be5c8c7f7d l=127.0.0.1:80 r=49.44.81.38:33024
2021-05-22 07:48:56.319 INFO    pyngrok.pro

100%|███████████████████████████████████████████| 15/15 [00:30<00:00,  2.01s/it]


In [40]:
import streamlit as st
st.sidebar.selectbox?

Signature:
st.sidebar.selectbox(
    label,
    options,
    index=0,
    format_func=<class 'str'>,
    key=None,
    help=None,
)
Docstring:
Display a select widget.

Parameters
----------
label : str
    A short label explaining to the user what this select widget is for.
options : list, tuple, numpy.ndarray, pandas.Series, or pandas.DataFrame
    Labels for the select options. This will be cast to str internally
    by default. For pandas.DataFrame, the first column is selected.
index : int
    The index of the preselected option on first render.
format_func : function
    Function to modify the display of the labels. It receives the option
    as an argument and its output will be cast to str.
key : str
    An optional string to use as the unique key for the widget.
    If this is omitted, a key will be generated for the widget
    based on its content. Multiple widgets of the same type may
    not share the same key.
help : str
    A tooltip that gets displayed next to the select